# Your first convolutional neural network

# imports

In [1]:
import numpy as np

from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

from tensorflow.keras.datasets import cifar10

2022-03-26 21:23:43.065443: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-26 21:23:43.065586: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# data

In [ ]:
NUM_CLASSES = 10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [ ]:
x_train[54, 12, 13, 1]

# architecture

In [ ]:
input_layer = Input(shape=(32, 32, 3))

conv_layer_1 = Conv2D(filters=10, kernel_size=(4, 4), strides=2, padding="same")(input_layer)

conv_layer_2 = Conv2D(filters=20, kernel_size=(3, 3), strides=2, padding="same")(conv_layer_1)

flatten_layer = Flatten()(conv_layer_2)

output_layer = Dense(units=10, activation="softmax")(flatten_layer)

model = Model(input_layer, output_layer)

In [ ]:
model.summary()

In [ ]:
input_layer = Input((32, 32, 3))

x = Conv2D(filters=32, kernel_size=3, strides=1, padding="same")(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


x = Conv2D(filters=32, kernel_size=3, strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


x = Conv2D(filters=64, kernel_size=3, strides=1, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


x = Conv2D(filters=64, kernel_size=3, strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


x = Flatten()(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate=0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation("softmax")(x)

model = Model(input_layer, output_layer)

In [ ]:
model.summary()

# train

In [ ]:
opt = Adam(lr=0.0005)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10, shuffle=True, validation_data=(x_test, y_test))

In [ ]:
model.layers[6].get_weights()

# analysis

In [ ]:
model.evaluate(x_test, y_test, batch_size=1000)

In [ ]:
CLASSES = np.array(["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"])

preds = model.predict(x_test)
preds_single = CLASSES[np.argmax(preds, axis=-1)]
actual_single = CLASSES[np.argmax(y_test, axis=-1)]

In [ ]:
import matplotlib.pyplot as plt

n_to_show = 10
indices = np.random.choice(range(len(x_test)), n_to_show)

fig = plt.figure(figsize=(15, 3))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i, idx in enumerate(indices):
    img = x_test[idx]
    ax = fig.add_subplot(1, n_to_show, i + 1)
    ax.axis("off")
    ax.text(0.5, -0.35, "pred = " + str(preds_single[idx]), fontsize=10, ha="center", transform=ax.transAxes)
    ax.text(0.5, -0.7, "act = " + str(actual_single[idx]), fontsize=10, ha="center", transform=ax.transAxes)
    ax.imshow(img)